In [2]:
import requests
url = "http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

In [4]:
res = requests.get(url,stream = True)
with open('aclImdb_v1.tar.gz',"wb") as f:
    for chunk in res.iter_content(chunk_size=1024):
        if chunk:
            f.write(chunk)

In [6]:
import tarfile
tfile = tarfile.open("aclImdb_v1.tar.gz",'r:gz')
result = tfile.extractall()

In [ ]:
from